In [3]:
%load_ext autoreload
%autoreload 2
from parse.parse_reactions import parse_reaction_string
from parse.parse_species import parse_species_string, parse_all_species
from parse.parse_input import separate_input_file, parse_args_kwargs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
sections = separate_input_file('input/sample_input.txt')
species_section = sections['species']
parse_all_species(species_section)



# for line in sections['species']:
#     print(parse_species_string(line))
# parse_args_kwargs(sections['species'])

{'general': [], 'species': [], 'rate constants': [], 'reactions': []}


([Species(name='I', type=<SpeciesType.UNIT: 1>, FW=50.0, c0=0.005),
  Species(name='R', type=<SpeciesType.UNIT: 1>, FW=25.0, c0=0.0),
  Species(name='A', type=<SpeciesType.UNIT: 1>, FW=100.0, c0=1.5),
  Species(name='B', type=<SpeciesType.UNIT: 1>, FW=200.0, c0=1.5),
  Species(name='P[R.A]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0),
  Species(name='P[R.B]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0),
  Species(name='P[A.A]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0),
  Species(name='P[A.B]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0),
  Species(name='P[B.A]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0),
  Species(name='P[B.B]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0),
  Species(name='D', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0)],
 [EquivalentSpecies(name='P[-.A]', type=<SpeciesType.POLY: 2>, eq='P[A.A]|P[B.A]'),
  EquivalentSpecies(name='P[-.B]', type=<SpeciesType.POLY: 2>, eq='P[A.B]|P[B.B]')])

In [5]:
species_string = 'U I      c0=0.005 FW=50.0'
parse_species_string(species_string)

Species(name='I', type=<SpeciesType.UNIT: 1>, FW=50.0, c0=0.005)

In [6]:
reaction_string = '23PAA + B -kpAAB-> 5PAB'
parse_reaction_string(reaction_string)

Number: 23
Remaining Part: PAA


TypeError: ReactionSpecies.__init__() missing 2 required positional arguments: 'poly_product1' and 'poly_product2'

In [ ]:
from itertools import product

# Given string format
template_string = 'P[-.A] + P[-.B] -kpABC-> D'

# Define possible replacements for "P[-.A]" and "P[-.B]"
replacements_A = ['P[A.A]', 'P[B.A]']
replacements_B = ['P[A.B]', 'P[B.B]']

# Generate all permutations
permutations = list(product(replacements_A, replacements_B))

# Create a list of strings with permutations
for p in permutations:
    print(p)
result_strings = [template_string.replace('P[-.A]', p[0]).replace('P[-.B]', p[1]) for p in permutations]

# Print the result
for idx, result_string in enumerate(result_strings, start=1):
    print(f'{idx}: {result_string}')


('P[A.A]', 'P[A.B]')
('P[A.A]', 'P[B.B]')
('P[B.A]', 'P[A.B]')
('P[B.A]', 'P[B.B]')
1: P[A.A] + P[A.B] -kpABC-> D
2: P[A.A] + P[B.B] -kpABC-> D
3: P[B.A] + P[A.B] -kpABC-> D
4: P[B.A] + P[B.B] -kpABC-> D
